In [1]:
import random

import pandas as pd
import numpy as np

def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val

Run Monte Carlo model in six month intervals, for each interval:

1.) Extrapolate country trends for all countries

2.) For each country, if 70% or more hens projected to be cage-free in that country, immediately become a NATIONAL law in that country

3.) Each year, expect a scientific report sometime between December 2022 and 2024… when report happens, there is a 36% chance of EU law after 2 years (+/-)

4.) The EU Presidency rotates through a known sequence (Germany now, Portugal in Jan 2021, Slovenia in July 2021, France, Czech Republic, Sweden, etc.) - if it hits a country that has a national law, 57% chance it becomes EU law after 12-36 months

In [2]:
N_SCENARIOS = 10000
scenario_results = []
scenario_reasons = []
scenario_countries = []

for n in range(N_SCENARIOS):
    EU_PRESIDENCIES = ['Portugal', 'Slovenia', 'France', 'Czechia', 'Sweden', 'Spain', 'Belgium',
                       'Hungary', 'Poland', 'Denmark', 'Cyprus', 'Ireland', 'Lithuania', 'Greece',
                       'Italy', 'Latvia', 'Luxembourg', 'Netherlands', 'Slovakia', 'Malta']
    COUNTRIES_W_BANS = ['Germany', 'Czechia', 'Luxembourg']
    COUNTRY_RATES = {'Portugal': 0.138,
                     'Slovenia': 0.607,
                     'France': 0.459,
                     'Czechia': 0.26,
                     'Sweden': 0.908,
                     'Spain': 0.232,
                     'Belgium': 0.628,
                     'Hungary': 0.297,
                     'Poland': 0.179,
                     'Denmark': 0.87,
                     'Cyprus': 0.412,
                     'Germany': 0.94,
                     'Ireland': 0.485,
                     'Lithuania': 0.111,
                     'Greece': 0.227,
                     'Italy': 0.506,
                     'Latvia': 0.183,
                     'Luxembourg': 1,
                     'Netherlands': 0.859,
                     'Slovakia': 0.167,
                     'Malta': 0.06}
    report = False
    ban_presidency = False
    done = False
    i = 1
    verbose = (n % 500 == 0)
    if verbose:
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
    for year in range(2021, 10000):
        for season in ['H1', 'H2']:
            if not done:
                tick = '{} {}'.format(year, season)
                if verbose:
                    print('{}: {}'.format(i, tick))

                # Extrapolate country trends for all countries
                for country in COUNTRY_RATES.keys():
                    COUNTRY_RATES[country] += 0.03 # TODO: Country-specific trends
                    if COUNTRY_RATES[country] > 1:
                        COUNTRY_RATES[country] = 1

                # For each country, if 70% or more hens projected to be cage-free in that country,
                # immediately become a NATIONAL law in that country
                for country in COUNTRY_RATES.keys():
                    if COUNTRY_RATES[country] >= 0.7:
                        if country not in COUNTRIES_W_BANS:
                            if verbose:
                                print('...{} passed a national ban!'.format(country))
                            COUNTRIES_W_BANS.append(country)

                # Each year, expect a scientific report sometime between December 2022 and 2024
                if not report and year >= 2023:
                    if random.random() < 0.25:
                        if verbose:
                            print('...The scientific report arrived!')
                        report = True
                        report_on_tick = i

                # when report happens, there is a 36% chance of EU law after 12-36 months
                if report:
                    if (i == report_on_tick >= 2 and random.random() < 0.5) or (i > report_on_tick + 6):
                        if random.random() < 0.36:
                            if verbose:
                                print('...EU BAN ACHIEVED (by scientific report)! 🎉🎉🎉')
                            scenario_results.append(year)
                            scenario_reasons.append('report')
                            scenario_countries.append('report')
                            done = True
                        else:
                            if verbose:
                                print('...Scientific report didn\'t lead to ban')

                # The EU Presidency rotates...
                president = EU_PRESIDENCIES[(i - 1) % len(EU_PRESIDENCIES)]
                if verbose:
                    print('...{} is President'.format(president))

                # TODO: if it hits a country that has a national law, 57% chance it becomes EU law after 12-36 months
                if president in COUNTRIES_W_BANS and not ban_presidency:
                    if verbose:
                        print('...The EU Presidency has a national ban, attempting EU ban')
                    ban_presidency = True
                    ban_presidency_on_tick = i
                    ban_country = president

                if ban_presidency:
                    if (i >= ban_presidency_on_tick + 2 and random.random() < 0.5) or (i > ban_presidency_on_tick + 6):
                        if random.random() < 0.57:
                            if verbose:
                                print('...EU BAN ACHIEVED (by {} EU Presidency)! 🎉🎉🎉'.format(ban_country))
                            scenario_results.append(year)
                            scenario_reasons.append('presidency')
                            scenario_countries.append(ban_country)
                            done = True
                        else:
                            if verbose:
                                print('...attempted EU ban by Presidency failed')
                            ban_presidency = False

                i += 1
                
    if verbose:
        print('-')

## SCENARIO 1/10000 ##
1: 2021 H1
...Sweden passed a national ban!
...Denmark passed a national ban!
...Netherlands passed a national ban!
...Portugal is President
2: 2021 H2
...Slovenia is President
3: 2022 H1
...Belgium passed a national ban!
...France is President
4: 2022 H2
...Slovenia passed a national ban!
...Czechia is President
...The EU Presidency has a national ban, attempting EU ban
5: 2023 H1
...Sweden is President
6: 2023 H2
...Spain is President
...EU BAN ACHIEVED (by Czechia EU Presidency)! 🎉🎉🎉
-
## SCENARIO 501/10000 ##
1: 2021 H1
...Sweden passed a national ban!
...Denmark passed a national ban!
...Netherlands passed a national ban!
...Portugal is President
2: 2021 H2
...Slovenia is President
3: 2022 H1
...Belgium passed a national ban!
...France is President
4: 2022 H2
...Slovenia passed a national ban!
...Czechia is President
...The EU Presidency has a national ban, attempting EU ban
5: 2023 H1
...Sweden is President
6: 2023 H2
...Spain is President
...EU BAN ACHIEVE

## SCENARIO 6501/10000 ##
1: 2021 H1
...Sweden passed a national ban!
...Denmark passed a national ban!
...Netherlands passed a national ban!
...Portugal is President
2: 2021 H2
...Slovenia is President
3: 2022 H1
...Belgium passed a national ban!
...France is President
4: 2022 H2
...Slovenia passed a national ban!
...Czechia is President
...The EU Presidency has a national ban, attempting EU ban
5: 2023 H1
...Sweden is President
6: 2023 H2
...Spain is President
...EU BAN ACHIEVED (by Czechia EU Presidency)! 🎉🎉🎉
-
## SCENARIO 7001/10000 ##
1: 2021 H1
...Sweden passed a national ban!
...Denmark passed a national ban!
...Netherlands passed a national ban!
...Portugal is President
2: 2021 H2
...Slovenia is President
3: 2022 H1
...Belgium passed a national ban!
...France is President
4: 2022 H2
...Slovenia passed a national ban!
...Czechia is President
...The EU Presidency has a national ban, attempting EU ban
5: 2023 H1
...The scientific report arrived!
...Scientific report didn't lead to

In [3]:
for y in range(2021, 2036):
    print('% chance by {}: {}'.format(y, print_pct(np.sum([s <= y for s in scenario_results]) / len(scenario_results), digits=1)))

% chance by 2021: <0.1%
% chance by 2022: <0.1%
% chance by 2023: 33.3%
% chance by 2024: 55.6%
% chance by 2025: 69.0%
% chance by 2026: 81.0%
% chance by 2027: 91.0%
% chance by 2028: 94.9%
% chance by 2029: 98.0%
% chance by 2030: 99.1%
% chance by 2031: 99.6%
% chance by 2032: 99.9%
% chance by 2033: >99.9%
% chance by 2034: >99.9%
% chance by 2035: >99.9%


In [4]:
pd.Series(scenario_reasons).value_counts(normalize=True) * 100

presidency    75.408261
report        24.591739
dtype: float64

In [5]:
pd.Series(scenario_countries).value_counts(normalize=True) * 100

Czechia        51.114313
report         24.591739
Belgium        10.259366
Denmark         9.567723
Cyprus          1.633045
Italy           1.623439
Ireland         0.758886
Netherlands     0.153698
Luxembourg      0.124880
Portugal        0.086455
Slovakia        0.057637
Slovenia        0.019212
France          0.009606
dtype: float64

In [6]:
# TODO: Can calculate chickens saved by EU ban by looking at delta with existing state at time of ban
# TODO: Can calculate highest-leverage opportunities